In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from collections import Counter
from PIL import Image

In [5]:
def load_data(image_dir):
    # Load all image files and split into train and test datasets
    classes = [d.name for d in os.scandir(image_dir) if d.is_dir()]
    file_paths = []
    labels = []

    for i, class_name in enumerate(classes):
        class_dir = os.path.join(image_dir, class_name)
        files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if f.endswith('.jpg')]
        file_paths.extend(files)
        labels.extend([i] * len(files))

    train_paths, test_paths, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=42, stratify=labels)
    return train_paths, test_paths, train_labels, test_labels, classes

In [6]:
def create_data_loaders(train_paths, test_paths, train_labels, test_labels):
    # Data transformations
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Custom dataset class
    class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, paths, labels, transform):
            self.paths = paths
            self.labels = labels
            self.transform = transform

        def __len__(self):
            return len(self.paths)

        def __getitem__(self, index):
            image = Image.open(self.paths[index]).convert('RGB')
            label = self.labels[index]
            return self.transform(image), label

    # Class weighting for imbalanced classes
    count = Counter(train_labels)
    class_weights = torch.tensor([1 / (count[i] + 1e-6) for i in range(len(count))], dtype=torch.float32)
    samples_weight = class_weights[train_labels]
    sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

    train_dataset = CustomDataset(train_paths, train_labels, train_transform)
    test_dataset = CustomDataset(test_paths, test_labels, test_transform)

    train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

    return train_loader, test_loader


In [7]:
# Model definition
def define_model(num_classes):
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256, num_classes)
    )
    return model



In [20]:
# Model training
def train_model(model, train_loader, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)

    for epoch in range(10):  # train for 10 epochs
        model.train()
        running_loss = 0.0
        start_time = time.time()

        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if (i + 1) % 20 == 0:
                print(f"Epoch {epoch+1}, Step {i+1}, Loss: {loss.item():.4f}")

        end_time = time.time()
        epoch_duration = end_time - start_time
        print(f'Epoch {epoch + 1} completed in {epoch_duration:.2f} seconds')



In [18]:
# Model evaluation
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

In [21]:
# Main function
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image_dir = '/content/drive/MyDrive/JPEGImages'
    train_paths, test_paths, train_labels, test_labels, classes = load_data(image_dir)
    train_loader, test_loader = create_data_loaders(train_paths, test_paths, train_labels, test_labels)
    model = define_model(len(classes))
    train_model(model, train_loader, device)
    evaluate_model(model, test_loader, device)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Step 20, Loss: 3.0218
Epoch 1, Step 40, Loss: 2.6686
Epoch 1, Step 60, Loss: 2.3348
Epoch 1, Step 80, Loss: 1.9937
Epoch 1, Step 100, Loss: 1.7222
Epoch 1, Step 120, Loss: 1.8468
Epoch 1 completed in 42.05 seconds
Epoch 2, Step 20, Loss: 1.8759
Epoch 2, Step 40, Loss: 1.4452
Epoch 2, Step 60, Loss: 1.5666
Epoch 2, Step 80, Loss: 1.4433
Epoch 2, Step 100, Loss: 1.4067
Epoch 2, Step 120, Loss: 1.4588
Epoch 2 completed in 41.54 seconds
Epoch 3, Step 20, Loss: 1.3716
Epoch 3, Step 40, Loss: 1.4406
Epoch 3, Step 60, Loss: 0.9859
Epoch 3, Step 80, Loss: 1.2322
Epoch 3, Step 100, Loss: 1.3057
Epoch 3, Step 120, Loss: 1.0770
Epoch 3 completed in 41.56 seconds
Epoch 4, Step 20, Loss: 0.9321
Epoch 4, Step 40, Loss: 1.0149
Epoch 4, Step 60, Loss: 0.9969
Epoch 4, Step 80, Loss: 1.1406
Epoch 4, Step 100, Loss: 1.1979
Epoch 4, Step 120, Loss: 0.8895
Epoch 4 completed in 41.61 seconds
Epoch 5, Step 20, Loss: 1.0547
Epoch 5, Step 40, Loss: 1.1319
Epoch 5, Step 60, Loss: 0.9568
Epoch 5, Step 8